In [6]:
import sys
import os
sys.path.insert(0, os.path.abspath("../.."))

# Searching the object store

In this example we will load in data from 3 sites. The data will be separated into sections and stored in the object store. We can then use a search function to search for data from the sites over a certain daterange, retrieve that data and visualize it.